https://www.kaggle.com/code/tanyan13/news-article-classification

In [1]:
import txtai

In [2]:
import pandas as pd

In [5]:
import json

In [3]:
dataset = r"D:\work\data\text\News_Category_Dataset_v3.json"

In [7]:
df = pd.read_json (dataset, lines = True)

In [9]:
import spacy

In [8]:
df

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [23]:
df = df.sample(1000).reset_index(drop=True)

In [24]:
df.to_parquet("text_data.parquet", index=False)

In [10]:
nlp = spacy.load("en_core_web_sm")

In [11]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

In [14]:
ents = [(e.text, e.label_) for e in doc.ents]

In [15]:
ents

[('Apple', 'ORG'), ('U.K.', 'GPE'), ('$1 billion', 'MONEY')]

In [19]:
from tqdm import tqdm
tqdm.pandas()

In [20]:
def do_ner(text):
    doc = nlp(text)
    ents = [(e.text, e.label_) for e in doc.ents]
    return ents

In [25]:
df['ents'] = df['headline'].progress_apply(do_ner)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 214.94it/s]


In [26]:
df['ents']

0                  [(9, CARDINAL), (Found Dead, PERSON)]
1                                                     []
2      [(9, CARDINAL), (Inspirational Fall Mantels, E...
3      [(Leonardo DiCaprio Fights, PERSON), (Revenant...
4              [(First Week On Crime And Voting, EVENT)]
                             ...                        
995    [(John Dingell, PERSON), (Longest Serving, ORG...
996                      [(Paul Walker Tragedy, PERSON)]
997    [(3, CARDINAL), (Pokemon Go', ORG), (Craze, PR...
998                                                   []
999    [(Inside DisneyWorld's, ORG), (Seven, CARDINAL...
Name: ents, Length: 1000, dtype: object

In [32]:
import uuid

In [35]:
df['docid'] = [f"doc://{str(uuid.uuid4())}" for x in range(len(df))]

In [37]:
df = df[df['ents'].str.len() > 0]

In [44]:
df_ent = df[['docid', 'ents']]

In [49]:
df_ent = df_ent.explode('ents')

In [50]:
df_ent

,docid,ents
0,doc://e7875d7d-e38d-4846-85a9-61ab4bbd02a6,"(9, CARDINAL)"
0,doc://e7875d7d-e38d-4846-85a9-61ab4bbd02a6,"(Found Dead, PERSON)"
2,doc://5b0060b8-d034-4831-b980-9d8cb89eb130,"(9, CARDINAL)"
2,doc://5b0060b8-d034-4831-b980-9d8cb89eb130,"(Inspirational Fall Mantels, EVENT)"
3,doc://8763e31d-0767-432f-9dff-ead47017ab15,"(Leonardo DiCaprio Fights, PERSON)"
...,...,...
997,doc://eb9488f5-afa7-4829-9a20-16b7727b22b3,"(Pokemon Go', ORG)"
997,doc://eb9488f5-afa7-4829-9a20-16b7727b22b3,"(Craze, PRODUCT)"
999,doc://7fd59fd0-1245-4a6b-a828-49b7857fd862,"(Inside DisneyWorld's, ORG)"
999,doc://7fd59fd0-1245-4a6b-a828-49b7857fd862,"(Seven, CARDINAL)"


In [54]:
df_ent[['ent', 'ent_type']] = pd.DataFrame(df_ent['ents'].tolist(), columns=['ent', 'ent_type'], index= df_ent.index)

In [57]:
df_ent.drop(column=['ents'])

TypeError: DataFrame.drop() got an unexpected keyword argument 'column'